In [ ]:
from pyspark.sql.types import LongType

In [ ]:
# Input data and output folders - SPARK SQL
inUsers = "Users.txt"
inSongs = "Songs.txt"
inListenToSongs = "ListenToSongs.txt"

outputPathPart1 = "outPart1SQL/"
outputPathPart2 = "outPart2SQL/"

In [ ]:
# Part 1

In [ ]:
# Read the input data
usersDF = spark.read.load(inUsers,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "UID")\
.withColumnRenamed("_c1", "Name")\
.withColumnRenamed("_c2", "Surname")\
.withColumnRenamed("_c3", "Gender")\
.withColumnRenamed("_c4", "YearOfBirth")

In [ ]:
songsDF = spark.read.load(inSongs,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "SID")\
.withColumnRenamed("_c1", "Title")\
.withColumnRenamed("_c2", "MusicGenre")

In [ ]:
listenDF = spark.read.load(inListenToSongs,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "SID")\
.withColumnRenamed("_c1", "UID")\
.withColumnRenamed("_c2", "StartTimestamp")\
.withColumnRenamed("_c3", "EndTimestamp")\
.cache()

In [ ]:
# Associate the input dataframes to temporary tables
usersDF.createOrReplaceTempView("users")
songsDF.createOrReplaceTempView("songs")
listenDF.createOrReplaceTempView("listen")

In [ ]:
# Select the songs never listened to by young users in the last two years
midsIncomesDF = spark.sql("""SELECT SID
FROM songs 
     LEFT ANTI JOIN
     (SELECT SID
      FROM listen, users 
      WHERE listen.UID=users.UID
      AND StartTimestamp>='2018/09/14' 
      AND StartTimestamp<='2020/09/13'
      AND YearOfBirth>1999) listenByYoung
     ON songs.SID=listenByYoung.SID""")

In [ ]:
#midsIncomesDF.show()

In [ ]:
# Store the result
midsIncomesDF.write.csv(outputPathPart1,header=False)

In [ ]:
# Part 2

In [ ]:
# Define a UDF that extract the year part from Date
def yearFunc(timestamp):
    return int(timestamp.split("/")[0])
    
spark.udf.register("yearFunc", yearFunc, LongType())

In [ ]:
# Compute for each song its popularity in each year, 
# i.e., the number of distinct users who listened to it in that year

In [ ]:
# Remove duplicates. We are interested in the number of distinct users for each (song,year)
sidYearDistinctUserDF = spark.sql("""
SELECT DISTINCT SID,
yearFunc(StartTimestamp) as Year,
UID 
FROM listen""")

In [ ]:
# Associate sidYearDistinctUserDF to a temporary table
sidYearDistinctUserDF.createOrReplaceTempView("listenNoDuplicates")

In [ ]:
# Count the number of distinct user for each pair (sid,year)
sidYearPopularityDF = spark.sql("""SELECT SID, Year, COUNT(*) as YearlyPopularity
FROM listenNoDuplicates
GROUP BY SID, Year""")

In [ ]:
#sidYearPopularityDF.show()

In [ ]:
# Associate sidYearPopularityDF to a temporary table
sidYearPopularityDF.createOrReplaceTempView("SIDYearlyPopularity")

In [ ]:
# Compute the highest yearly popularity for each song
sidMaxPopDF = spark.sql("""SELECT SID, MAX(YearlyPopularity) as MaxPop
FROM SIDYearlyPopularity
GROUP BY SID""")

In [ ]:
#sidMaxPopDF.show()

In [ ]:
# Associate sidMaxPopDF to a temporary table
sidMaxPopDF.createOrReplaceTempView("SIDMaxYearlyPopularity")

In [ ]:
# Select for each song the first year 
# associated with its highest popularity
sidFirstYearMaxPopDF = spark.sql("""SELECT SIDYearlyPopularity.SID, MIN(Year) 
FROM SIDYearlyPopularity,SIDMaxYearlyPopularity
WHERE SIDYearlyPopularity.SID=SIDMaxYearlyPopularity.SID
AND SIDYearlyPopularity.YearlyPopularity=SIDMaxYearlyPopularity.MaxPop
GROUP BY SIDYearlyPopularity.SID""")

In [ ]:
#sidFirstYearMaxPopDF.show()

In [ ]:
# Store the result
sidFirstYearMaxPopDF.write.csv(outputPathPart2,header=False)